In [2]:
import os
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct

URL = os.getenv("QDRANT_URL")
API_KEY = os.getenv("QDRANT_API_KEY")
COLLECTION_NAME = os.getenv("QDRANT_COLLECTION_NAME")

client = QdrantClient(url=URL, api_key=API_KEY)

# Add multiple keys in payload
points = [
    PointStruct(
        id=1,
        vector=[0.1, 0.2, 0.3],  # Example vector
        payload={
            "category": "news",
            "timestamp": "2024-12-30T10:00:00Z",
            "author": "Alice",
        },
    ),
    PointStruct(
        id=2,
        vector=[0.4, 0.5, 0.6],  # Example vector
        payload={
            "category": "blog",
            "timestamp": "2024-12-29T15:30:00Z",
            "author": "Bob",
        },
    ),
]

# Upsert points into Qdrant
client.upsert(collection_name="test", points=points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [5]:
from qdrant_client.models import Filter, FieldCondition, MatchValue

# Create a filter to search by multiple keys
filter_query = Filter(
    must=[
        FieldCondition(key="category", match=MatchValue(value="news")),
        FieldCondition(key="author", match=MatchValue(value="Alice")),
    ]
)

# Search with the filter
result = client.search(
    collection_name="test",
    query_vector=[0.1, 0.2, 0.3],  # Query vector
)

In [1]:
from groq import Groq

client = Groq()
completion = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[
        {"role": "user", "content": "hello"},
        {
            "role": "assistant",
            "content": "Hello. It's nice to meet you. Is there something I can help you with or would you like to chat?",
        },
    ],
    temperature=1,
    max_completion_tokens=1024,
    top_p=1,
    stream=True,
    stop=None,
)

for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")

In [18]:
from typing import Optional, List
from typing_extensions import TypedDict


class ExpenseSchema(TypedDict):
    """
    Schema for representing an expense.
    Attributes:
        id (str): Unique identifier for the expense.
        vector (List[float]): A list of numerical values associated with the expense.
        date (str): The date of the expense in YYYY-MM-DD format.
        amount (float): The amount of money spent in float format.
        category (str): The category of the expense (e.g., food, travel, etc.).
        description (Optional[str]): A brief description of the expense (optional).
    """

    id: str
    vector: List[float]
    date: str
    amount: float
    category: str
    description: Optional[str]


class ExpenseCreate(ExpenseSchema):
    pass


tools = [ExpenseSchema]

In [38]:
from langchain_groq import ChatGroq
from PIL import Image
import io, os
import base64

image_path = "test_inv.png"
with open(image_path, "rb") as image_file:
    image_data = base64.b64encode(image_file.read()).decode("utf-8")

API_KEY = os.getenv("GROQ_API_KEY")

llm_vision = ChatGroq(
    api_key=API_KEY,
    model="llama-3.2-11b-vision-preview",
)

# Extract data from the image using the vision model with a text message
response = llm_vision.with_structured_output(ExpenseSchema).invoke(input=image_data)

# Print the extracted data
print(response)

{'id': '12345', 'vector': [1.0, 2.0, 3.0], 'date': '2022-01-01', 'amount': 100.0, 'category': 'food', 'description': 'Lunch at a restaurant'}


In [30]:
# Ensure the necessary imports are present
from langchain_groq import ChatGroq
from app.schemas import ExpenseSchema

# Define the image URL
IMAGE_DATA_URL = "https://templates.invoicehome.com/invoice-template-en-neat-750px.png"

# Initialize the ChatGroq model
llm_vision = ChatGroq(
    model="llama-3.2-90b-vision-preview",
)

# Define the input for the model
input_data = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "only extract total expense as a list nothing extra:\n    date: str\n    amount: float\n    category: str\n    description: Optional[str]",
            },
            {
                "type": "image_url",
                "image_url": {"url": IMAGE_DATA_URL},
            },
        ],
    },
]

# Invoke the model with structured output
completion = llm_vision.with_structured_output(ExpenseSchema).invoke(input=input_data)

# Print the extracted data
print(completion)

{'amount': 204.75, 'date': '11/02/2010', 'description': 'None', 'category': 'INVOICE'}
